In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('../2_data_preparation/cleaned_data.csv')
clustered = pd.read_csv('data_clustered.csv')

In [3]:
clustered = clustered[['name', 'cluster']]
clustered

,name,cluster
0,Lerone Murphy,4
1,Edson Barboza,4
2,Khaos Williams,4
3,Carlston Harris,2
4,Themba Gorimbo,2
...,...,...
2391,Nick Sanzo,3
2392,Jim Mullen,3
2393,Yoshiki Takahashi,1
2394,Wallid Ismail,1


In [4]:
data['winner'].value_counts()

winner
f1    7289
0      111
Name: count, dtype: int64

In [5]:
# filter data where clear decision
data = data[data['winner'] == 'f1']
data['winner'].value_counts()

winner
f1    7289
Name: count, dtype: int64

In [6]:
data

,end_time,f1,f2,fight_date,method,referee,round,time_format,weightclass,winner,f1_body,f1_clinch,f1_control,f1_distance,f1_dob,f1_ground,f1_head,f1_height,f1_kd,f1_leg,f1_reach,f1_record,f1_rev,f1_sapm,f1_sig_str,f1_sig_str_%,f1_slpm,f1_stance,f1_str__acc_,f1_str__def,f1_sub__avg_,f1_sub_att,f1_td,f1_td_%,f1_td_acc_,f1_td_avg_,f1_td_def_,f1_total_str,f1_weight,f2_body,f2_clinch,f2_control,f2_distance,f2_dob,f2_ground,f2_head,f2_height,f2_kd,f2_leg,f2_reach,f2_record,f2_rev,f2_sapm,f2_sig_str,f2_sig_str_%,f2_slpm,f2_stance,f2_str__acc_,f2_str__def,f2_sub__avg_,f2_sub_att,f2_td,f2_td_%,f2_td_acc_,f2_td_avg_,f2_td_def_,f2_total_str,f2_weight,gender,year
0,5:00,Lerone Murphy,Edson Barboza,2024-05-18,Decision - Unanimous,Herb Dean,5,5,featherweight,f1,27 of 34,13 of 17,5:19,205 of 344,"Jul 22, 1991",2 of 3,172 of 308,175.26,0,21 of 22,185.42,Record: 14-0-1,0,2.60,220 of 364,60%,5.01,Orthodox,54%,60%,0.6,0,4 of 6,66%,52%,1.59,46%,259 of 410,65.77,15 of 21,2 of 3,5:19,77 of 239,"Jan 21, 1986",0 of 0,38 of 189,180.34,0,26 of 32,190.50,Record: 24-12-0,0,4.65,79 of 242,32%,4.11,Orthodox,44%,56%,0.1,0,0 of 0,0,50%,0.45,72%,79 of 242,65.77,M,2024
1,1:30,Khaos Williams,Carlston Harris,2024-05-18,KO/TKO,Dan Miragliotta,1,3,welterweight,f1,2 of 4,0 of 0,0:01,11 of 19,"Mar 30, 1994",1 of 1,5 of 9,182.88,1,5 of 7,195.58,Record: 15-3-0,0,5.42,12 of 20,60%,5.81,Orthodox,39%,42%,0.0,0,0 of 0,0,0%,0.00,80%,12 of 20,77.11,1 of 2,0 of 0,0:01,14 of 30,"Jul 09, 1987",0 of 0,10 of 25,182.88,0,3 of 3,193.04,Record: 19-6-0,0,2.66,14 of 30,46%,3.18,Orthodox,49%,52%,0.8,0,0 of 0,0,29%,1.97,55%,14 of 30,77.11,M,2024
2,5:00,Themba Gorimbo,Ramiz Brahimaj,2024-05-18,Decision - Unanimous,Mark Smith,3,3,welterweight,f1,31 of 35,35 of 38,10:13,6 of 10,"Jan 23, 1991",2 of 4,9 of 13,185.42,0,3 of 4,195.58,Record: 13-4-0,1,1.08,43 of 52,82%,2.53,Orthodox,69%,46%,0.0,0,5 of 6,83%,58%,4.04,78%,110 of 134,77.11,3 of 4,3 of 4,10:13,0 of 2,"Nov 17, 1992",0 of 0,0 of 2,177.80,0,0 of 0,182.88,Record: 10-5-0,0,3.69,3 of 6,50%,1.74,Orthodox,41%,45%,1.6,0,1 of 8,12%,35%,1.61,44%,15 of 20,77.11,M,2024
3,2:47,Adrian Yanez,Vinicius Salvador,2024-05-18,KO/TKO,Chris Tognoni,1,3,bantamweight,f1,1 of 2,0 of 0,0:17,15 of 28,"Nov 29, 1993",21 of 28,27 of 45,170.18,1,8 of 9,177.80,Record: 17-5-0,0,5.75,36 of 56,64%,6.51,Orthodox,41%,57%,0.0,0,0 of 0,0,0%,0.00,100%,36 of 58,61.23,6 of 6,0 of 0,0:17,17 of 34,"Jul 24, 1996",0 of 0,6 of 20,170.18,0,5 of 8,177.80,Record: 14-7-0,0,6.50,17 of 34,50%,5.48,Southpaw,43%,48%,0.0,0,0 of 0,0,11%,0.36,86%,17 of 34,61.23,M,2024
4,4:12,Angela Hill,Luana Pinheiro,2024-05-18,Submission,Herb Dean,2,3,strawweight,f1,9 of 15,3 of 3,3:06,21 of 71,"Jan 12, 1985",4 of 5,19 of 64,160.02,0,0 of 0,162.56,Record: 17-13-0,1,4.86,28 of 79,35%,5.38,Orthodox,49%,61%,0.1,1,2 of 6,33%,33%,0.80,76%,39 of 91,52.16,3 of 6,2 of 6,3:06,15 of 54,"Nov 18, 1992",0 of 0,11 of 51,157.48,0,3 of 3,157.48,Record: 11-3-0,0,4.17,17 of 60,28%,3.57,Orthodox,40%,63%,0.2,0,1 of 7,14%,33%,2.24,66%,25 of 77,52.16,M,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7395,0:39,Jerry Bohlander,Nick Sanzo,1997-02-07,Submission,John McCarthy,1,1,lightweight,f1,0 of 0,2 of 2,0,0 of 2,"Feb 12, 1974",0 of 0,2 of 3,180.34,0,0 of 1,185.29,Record: 11-4-0,0,0.00,2 of 4,50%,0.00,Orthodox,0%,0%,0.0,1,1 of 1,100%,0%,0.00,0%,3 of 5,90.26,0 of 0,0 of 0,0,0 of 0,NaN,0 of 0,0 of 0,175.26,0,0 of 0,179.62,Record: 1-1-0,0,0.00,0 of 0,0,0.00,Orthodox,0%,0%,0.0,0,0 of 1,0%,0%,0.00,0%,0 of 0,86.18,M,1997
7396,1:17,Vitor Belfort,Tra Telligman,1997-02-07,KO/TKO,John McCarthy,1,1,heavyweight,f1,0 of 0,5 of 15,0,2 of 4,"Apr 01, 1977",10 of 10,17 of 29,182.88,1,0 of 0,187.96,Record: 26-14-0 (1 NC),0,2.83,17 of 29,58%,1.59,Southpaw,46%,50%,0.6,0,0 of 0,0,60%,1.02,53%,23 of 35,83.91,1 of

In [7]:

# Merge the cluster column from clustered DataFrame based on fighter names (f1 and f2)
data = pd.merge(data, clustered[['name', 'cluster']], left_on='f1', right_on='name', how='left')
data.rename(columns={'cluster': 'winner_cluster'}, inplace=True)

data = pd.merge(data, clustered[['name', 'cluster']], left_on='f2', right_on='name', how='left')
data.rename(columns={'cluster': 'loser_cluster'}, inplace=True)

In [8]:
odds_df = data[['winner_cluster', 'loser_cluster']]

In [9]:
odds_df.head(2)

,winner_cluster,loser_cluster
0,4,4
1,4,2


In [10]:
odds_df['winner_cluster'].value_counts()

winner_cluster
2    3574
4    2981
1     643
0      65
3      26
Name: count, dtype: int64

In [11]:
import matplotlib.cm as cm

# Group by winner and loser clusters to count the number of wins and losses
win_counts = odds_df.groupby(['winner_cluster', 'loser_cluster']).size().unstack(fill_value=0)
loss_counts = odds_df.groupby(['loser_cluster', 'winner_cluster']).size().unstack(fill_value=0)

# Calculate win percentage by cluster matchup
win_percentage = (win_counts / (win_counts + loss_counts)) * 100
# Round win percentages to nearest integer
win_percentage_rounded = win_percentage.round()

# Convert rounded win percentages to integers
win_percentage_int = win_percentage_rounded.astype(int)

# Display win percentages by cluster matchup
win_percentage_rounded.style.background_gradient(cmap='Blues', axis=None)

loser_cluster,0,1,2,3,4
winner_cluster,,,,,
0,50.000000,36.000000,45.000000,100.000000,37.000000
1,64.000000,50.000000,28.000000,87.000000,26.000000
2,55.000000,72.000000,50.000000,93.000000,48.000000
3,0.000000,13.000000,7.000000,50.000000,0.000000
4,63.000000,74.000000,52.000000,100.000000,50.000000


In [12]:
(win_counts + loss_counts)

loser_cluster,0,1,2,3,4
winner_cluster,,,,,
0,2,25,94,2,30
1,25,296,1057,15,684
2,94,1057,3520,54,1984
3,2,15,54,40,5
4,30,684,1984,5,2820


In [13]:
data

,end_time,f1,f2,fight_date,method,referee,round,time_format,weightclass,winner,f1_body,f1_clinch,f1_control,f1_distance,f1_dob,f1_ground,f1_head,f1_height,f1_kd,f1_leg,f1_reach,f1_record,f1_rev,f1_sapm,f1_sig_str,f1_sig_str_%,f1_slpm,f1_stance,f1_str__acc_,f1_str__def,f1_sub__avg_,f1_sub_att,f1_td,f1_td_%,f1_td_acc_,f1_td_avg_,f1_td_def_,f1_total_str,f1_weight,f2_body,f2_clinch,f2_control,f2_distance,f2_dob,f2_ground,f2_head,f2_height,f2_kd,f2_leg,f2_reach,f2_record,f2_rev,f2_sapm,f2_sig_str,f2_sig_str_%,f2_slpm,f2_stance,f2_str__acc_,f2_str__def,f2_sub__avg_,f2_sub_att,f2_td,f2_td_%,f2_td_acc_,f2_td_avg_,f2_td_def_,f2_total_str,f2_weight,gender,year,name_x,winner_cluster,name_y,loser_cluster
0,5:00,Lerone Murphy,Edson Barboza,2024-05-18,Decision - Unanimous,Herb Dean,5,5,featherweight,f1,27 of 34,13 of 17,5:19,205 of 344,"Jul 22, 1991",2 of 3,172 of 308,175.26,0,21 of 22,185.42,Record: 14-0-1,0,2.60,220 of 364,60%,5.01,Orthodox,54%,60%,0.6,0,4 of 6,66%,52%,1.59,46%,259 of 410,65.77,15 of 21,2 of 3,5:19,77 of 239,"Jan 21, 1986",0 of 0,38 of 189,180.34,0,26 of 32,190.50,Record: 24-12-0,0,4.65,79 of 242,32%,4.11,Orthodox,44%,56%,0.1,0,0 of 0,0,50%,0.45,72%,79 of 242,65.77,M,2024,Lerone Murphy,4,Edson Barboza,4
1,1:30,Khaos Williams,Carlston Harris,2024-05-18,KO/TKO,Dan Miragliotta,1,3,welterweight,f1,2 of 4,0 of 0,0:01,11 of 19,"Mar 30, 1994",1 of 1,5 of 9,182.88,1,5 of 7,195.58,Record: 15-3-0,0,5.42,12 of 20,60%,5.81,Orthodox,39%,42%,0.0,0,0 of 0,0,0%,0.00,80%,12 of 20,77.11,1 of 2,0 of 0,0:01,14 of 30,"Jul 09, 1987",0 of 0,10 of 25,182.88,0,3 of 3,193.04,Record: 19-6-0,0,2.66,14 of 30,46%,3.18,Orthodox,49%,52%,0.8,0,0 of 0,0,29%,1.97,55%,14 of 30,77.11,M,2024,Khaos Williams,4,Carlston Harris,2
2,5:00,Themba Gorimbo,Ramiz Brahimaj,2024-05-18,Decision - Unanimous,Mark Smith,3,3,welterweight,f1,31 of 35,35 of 38,10:13,6 of 10,"Jan 23, 1991",2 of 4,9 of 13,185.42,0,3 of 4,195.58,Record: 13-4-0,1,1.08,43 of 52,82%,2.53,Orthodox,69%,46%,0.0,0,5 of 6,83%,58%,4.04,78%,110 of 134,77.11,3 of 4,3 of 4,10:13,0 of 2,"Nov 17, 1992",0 of 0,0 of 2,177.80,0,0 of 0,182.88,Record: 10-5-0,0,3.69,3 of 6,50%,1.74,Orthodox,41%,45%,1.6,0,1 of 8,12%,35%,1.61,44%,15 of 20,77.11,M,2024,Themba Gorimbo,2,Ramiz Brahimaj,2
3,2:47,Adrian Yanez,Vinicius Salvador,2024-05-18,KO/TKO,Chris Tognoni,1,3,bantamweight,f1,1 of 2,0 of 0,0:17,15 of 28,"Nov 29, 1993",21 of 28,27 of 45,170.18,1,8 of 9,177.80,Record: 17-5-0,0,5.75,36 of 56,64%,6.51,Orthodox,41%,57%,0.0,0,0 of 0,0,0%,0.00,100%,36 of 58,61.23,6 of 6,0 of 0,0:17,17 of 34,"Jul 24, 1996",0 of 0,6 of 20,170.18,0,5 of 8,177.80,Record: 14-7-0,0,6.50,17 of 34,50%,5.48,Southpaw,43%,48%,0.0,0,0 of 0,0,11%,0.36,86%,17 of 34,61.23,M,2024,Adrian Yanez,4,Vinicius Salvador,4
4,4:12,Angela Hill,Luana Pinheiro,2024-05-18,Submission,Herb Dean,2,3,strawweight,f1,9 of 15,3 of 3,3:06,21 of 71,"Jan 12, 1985",4 of 5,19 of 64,160.02,0,0 of 0,162.56,Record: 17-13-0,1,4.86,28 of 79,35%,5.38,Orthodox,49%,61%,0.1,1,2 of 6,33%,33%,0.80,76%,39 of 91,52.16,3 of 6,2 of 6,3:06,15 of 54,"Nov 18, 1992",0 of 0,11 of 51,157.48,0,3 of 3,157.48,Record: 11-3-0,0,4.17,17 of 60,28%,3.57,Orthodox,40%,63%,0.2,0,1 of 7,14%,33%,2.24,66%,25 of 77,52.16,M,2024,Angela Hill,4,Luana Pinheiro,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7284,0:39,Jerry Bohlander,Nick Sanzo,1997-02-07,Submission,John McCarthy,1,1,lightweight,f1,0 of 0,2 of 2,0,0 of 2,"Feb 12, 1974",0 of 0,2 of 3,180.34,0,0 of 1,185.29,Record: 11-4-0,0,0.00,2 of 4,50%,0.00,Orthodox,0%,0%,0.0,1,1 of 1,100%,0%,0.00,0%,3 of 5,90.26,0 of 0,0 of 0,0,0 of 0,NaN,0 of 0,0 of 0,175.26,0,0 of 0,179.62,Record: 1-1-0,0,0.00,0 of 0,0,0.00,Orthodox,0%,0%,0.0,0,0 of 1,0%,0%,0.00,0%,0 of 0,86.18,M,1997,Jerry Bohlander,3,Nick Sanzo,3
7285,1:17,Vitor Belfort,

In [14]:
data.groupby(['weightclass', 'winner_cluster']).size().unstack(fill_value=0)

winner_cluster,0,1,2,3,4
weightclass,,,,,
bantamweight,11,44,332,0,449
featherweight,3,67,316,0,363
flyweight,3,33,210,0,300
heavyweight,3,109,334,11,224
light heavyweight,2,70,374,0,217
lightweight,12,88,701,10,468
middleweight,15,103,535,5,329
strawweight,2,25,77,0,193
welterweight,14,104,695,0,438


In [15]:
data['weightclass'].value_counts()

weightclass
lightweight          1279
welterweight         1251
middleweight          987
bantamweight          836
featherweight         749
heavyweight           681
light heavyweight     663
flyweight             546
strawweight           297
Name: count, dtype: int64

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to calculate win percentages for a given weight class
def calculate_win_percentage(df):
    win_counts = df.groupby(['winner_cluster', 'loser_cluster']).size().unstack(fill_value=0)
    loss_counts = df.groupby(['loser_cluster', 'winner_cluster']).size().unstack(fill_value=0)
    win_percentage = (win_counts / (win_counts + loss_counts)) * 100
    return win_percentage.round().fillna(0)  # Round and fill NaN with 0

# List of unique weight classes
weight_classes = data['weightclass'].unique()

# Create a dictionary to store win percentages for each weight class
win_percentage_by_weight_class = {}

# Calculate win percentages for each weight class
for weight_class in weight_classes:
    df_filtered = data[data['weightclass'] == weight_class]
    win_percentage_by_weight_class[weight_class] = calculate_win_percentage(df_filtered)

# # Plot each heatmap individually
# for weight_class, win_percentage in win_percentage_by_weight_class.items():
#     plt.figure(figsize=(6, 4))
#     sns.heatmap(win_percentage, annot=True, cmap="Blues", fmt="g", cbar=True)
#     plt.title(f'Win Percentages for Weight Class: {weight_class}')
#     plt.xlabel('Loser Cluster')
#     plt.ylabel('Winner Cluster')
#     plt.show()


In [17]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to calculate win percentages for a given weight class
def calculate_win_percentage(df):
    win_counts = df.groupby(['winner_cluster', 'loser_cluster']).size().unstack(fill_value=0)
    loss_counts = df.groupby(['loser_cluster', 'winner_cluster']).size().unstack(fill_value=0)
    win_percentage = (win_counts / (win_counts + loss_counts)) * 100
    return win_percentage.round().fillna(0)  # Round and fill NaN with 0

# List of unique weight classes in the desired order
weight_classes_ordered = ['strawweight', 'flyweight', 'bantamweight', 'featherweight', 
                          'lightweight', 'welterweight', 'middleweight', 
                          'light heavyweight', 'heavyweight']

# Create a dictionary to store win percentages DataFrames for each weight class
win_percentage_df_by_weight_class = {}

# Calculate win percentages for each weight class in the specified order
for weight_class in weight_classes_ordered:
    df_filtered = data[data['weightclass'] == weight_class]
    win_percentage_df_by_weight_class[weight_class] = calculate_win_percentage(df_filtered)

# Display the win percentages as styled DataFrames with background gradient
for weight_class, win_percentage_df in win_percentage_df_by_weight_class.items():
    print(f'Win Percentages for Weight Class: {weight_class}')
    display(win_percentage_df.style.background_gradient(cmap='Blues', axis=None))
    print('\n' + '='*50 + '\n')


Win Percentages for Weight Class: strawweight


loser_cluster,0,1,2,4
winner_cluster,,,,
0,0.000000,0.000000,100.000000,0.000000
1,0.000000,50.000000,41.000000,36.000000
2,0.000000,59.000000,50.000000,57.000000
4,100.000000,64.000000,43.000000,50.000000




Win Percentages for Weight Class: flyweight


loser_cluster,0,1,2,4
winner_cluster,,,,
0,0.000000,0.000000,100.000000,33.000000
1,0.000000,50.000000,21.000000,20.000000
2,0.000000,79.000000,50.000000,48.000000
4,67.000000,80.000000,52.000000,50.000000




Win Percentages for Weight Class: bantamweight


...

loser_cluster,0,1,2,3,4
winner_cluster,,,,,
0,0.000000,25.000000,50.000000,100.000000,0.000000
1,75.000000,50.000000,30.000000,75.000000,26.000000
2,50.000000,70.000000,50.000000,100.000000,44.000000
3,0.000000,25.000000,0.000000,50.000000,0.000000
4,100.000000,74.000000,56.000000,100.000000,50.000000




Win Percentages for Weight Class: light heavyweight


loser_cluster,0,1,2,3,4
0,0.000000,50.000000,14.000000,0.000000,0.000000
1,50.000000,50.000000,27.000000,0.000000,36.000000
2,86.000000,73.000000,50.000000,0.000000,45.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000
4,100.000000,64.000000,55.000000,0.000000,50.000000




Win Percentages for Weight Class: heavyweight


loser_cluster,0,1,2,3,4
winner_cluster,,,,,
0,50.000000,0.000000,25.000000,100.000000,0.000000
1,100.000000,50.000000,34.000000,100.000000,35.000000
2,75.000000,66.000000,50.000000,92.000000,54.000000
3,0.000000,0.000000,8.000000,50.000000,0.000000
4,100.000000,65.000000,46.000000,0.000000,50.000000


In [18]:
# Group by winner and loser clusters to count the number of wins and losses
win_counts = data.groupby(['winner_cluster', 'loser_cluster']).size().unstack(fill_value=0)
loss_counts = data.groupby(['loser_cluster', 'winner_cluster']).size().unstack(fill_value=0)

# Calculate win percentage by cluster matchup
win_percentage = (win_counts / (win_counts + loss_counts)) * 100
# Round win percentages to nearest integer
win_percentage_rounded = win_percentage.round().fillna(0)  # Fill NaN with 0 for cases with no matches

# Reset index and melt the DataFrame to make it long-form
win_percentage_melted = win_percentage_rounded.reset_index().melt(id_vars='winner_cluster', var_name='loser_cluster', value_name='win_percentage')

# Merge the win percentage data with the original data
data_with_win_percentages = pd.merge(
    data, 
    win_percentage_melted, 
    how='left', 
    left_on=['winner_cluster', 'loser_cluster'], 
    right_on=['winner_cluster', 'loser_cluster']
)

# Check the merged DataFrame
display(data_with_win_percentages.head())

# Save the DataFrame to a CSV file for visualization in Tableau
data_with_win_percentages.to_csv('data_with_win_percentages.csv', index=False)


,end_time,f1,f2,fight_date,method,referee,round,time_format,weightclass,winner,f1_body,f1_clinch,f1_control,f1_distance,f1_dob,f1_ground,f1_head,f1_height,f1_kd,f1_leg,f1_reach,f1_record,f1_rev,f1_sapm,f1_sig_str,f1_sig_str_%,f1_slpm,f1_stance,f1_str__acc_,f1_str__def,f1_sub__avg_,f1_sub_att,f1_td,f1_td_%,f1_td_acc_,f1_td_avg_,f1_td_def_,f1_total_str,f1_weight,f2_body,f2_clinch,f2_control,f2_distance,f2_dob,f2_ground,f2_head,f2_height,f2_kd,f2_leg,f2_reach,f2_record,f2_rev,f2_sapm,f2_sig_str,f2_sig_str_%,f2_slpm,f2_stance,f2_str__acc_,f2_str__def,f2_sub__avg_,f2_sub_att,f2_td,f2_td_%,f2_td_acc_,f2_td_avg_,f2_td_def_,f2_total_str,f2_weight,gender,year,name_x,winner_cluster,name_y,loser_cluster,win_percentage
0,5:00,Lerone Murphy,Edson Barboza,2024-05-18,Decision - Unanimous,Herb Dean,5,5,featherweight,f1,27 of 34,13 of 17,5:19,205 of 344,"Jul 22, 1991",2 of 3,172 of 308,175.26,0,21 of 22,185.42,Record: 14-0-1,0,2.60,220 of 364,60%,5.01,Orthodox,54%,60%,0.6,0,4 of 6,66%,52%,1.59,46%,259 of 410,65.77,15 of 21,2 of 3,5:19,77 of 239,"Jan 21, 1986",0 of 0,38 of 189,180.34,0,26 of 32,190.50,Record: 24-12-0,0,4.65,79 of 242,32%,4.11,Orthodox,44%,56%,0.1,0,0 of 0,0,50%,0.45,72%,79 of 242,65.77,M,2024,Lerone Murphy,4,Edson Barboza,4,50.0
1,1:30,Khaos Williams,Carlston Harris,2024-05-18,KO/TKO,Dan Miragliotta,1,3,welterweight,f1,2 of 4,0 of 0,0:01,11 of 19,"Mar 30, 1994",1 of 1,5 of 9,182.88,1,5 of 7,195.58,Record: 15-3-0,0,5.42,12 of 20,60%,5.81,Orthodox,39%,42%,0.0,0,0 of 0,0,0%,0.00,80%,12 of 20,77.11,1 of 2,0 of 0,0:01,14 of 30,"Jul 09, 1987",0 of 0,10 of 25,182.88,0,3 of 3,193.04,Record: 19-6-0,0,2.66,14 of 30,46%,3.18,Orthodox,49%,52%,0.8,0,0 of 0,0,29%,1.97,55%,14 of 30,77.11,M,2024,Khaos Williams,4,Carlston Harris,2,52.0
2,5:00,Themba Gorimbo,Ramiz Brahimaj,2024-05-18,Decision - Unanimous,Mark Smith,3,3,welterweight,f1,31 of 35,35 of 38,10:13,6 of 10,"Jan 23, 1991",2 of 4,9 of 13,185.42,0,3 of 4,195.58,Record: 13-4-0,1,1.08,43 of 52,82%,2.53,Orthodox,69%,46%,0.0,0,5 of 6,83%,58%,4.04,78%,110 of 134,77.11,3 of 4,3 of 4,10:13,0 of 2,"Nov 17, 1992",0 of 0,0 of 2,177.80,0,0 of 0,182.88,Record: 10-5-0,0,3.69,3 of 6,50%,1.74,Orthodox,41%,45%,1.6,0,1 of 8,12%,35%,1.61,44%,15 of 20,77.11,M,2024,Themba Gorimbo,2,Ramiz Brahimaj,2,50.0
3,2:47,Adrian Yanez,Vinicius Salvador,2024-05-18,KO/TKO,Chris Tognoni,1,3,bantamweight,f1,1 of 2,0 of 0,0:17,15 of 28,"Nov 29, 1993",21 of 28,27 of 45,170.18,1,8 of 9,177.80,Record: 17-5-0,0,5.75,36 of 56,64%,6.51,Orthodox,41%,57%,0.0,0,0 of 0,0,0%,0.00,100%,36 of 58,61.23,6 of 6,0 of 0,0:17,17 of 34,"Jul 24, 1996",0 of 0,6 of 20,170.18,0,5 of 8,177.80,Record: 14-7-0,0,6.50,17 of 34,50%,5.48,Southpaw,43%,48%,0.0,0,0 of 0,0,11%,0.36,86%,17 of 34,61.23,M,2024,Adrian Yanez,4,Vinicius Salvador,4,50.0
4,4:12,Angela Hill,Luana Pinheiro,2024-05-18,Submission,Herb Dean,2,3,strawweight,f1,9 of 15,3 of 3,3:06,21 of 71,"Jan 12, 1985",4 of 5,19 of 64,160.02,0,0 of 0,162.56,Record: 17-13-0,1,4.86,28 of 79,35%,5.38,Orthodox,49%,61%,0.1,1,2 of 6,33%,33%,0.80,76%,39 of 91,52.16,3 of 6,2 of 6,3:06,15 of 54,"Nov 18, 1992",0 of 0,11 of 51,157.48,0,3 of 3,157.48,Record: 11-3-0,0,4.17,17 of 60,28%,3.57,Orthodox,40%,63%,0.2,0,1 of 7,14%,33%,2.24,66%,25 of 77,52.16,M,2024,Angela Hill,4,Luana Pinheiro,4,50.0


In [19]:
data_with_win_percentages

,end_time,f1,f2,fight_date,method,referee,round,time_format,weightclass,winner,f1_body,f1_clinch,f1_control,f1_distance,f1_dob,f1_ground,f1_head,f1_height,f1_kd,f1_leg,f1_reach,f1_record,f1_rev,f1_sapm,f1_sig_str,f1_sig_str_%,f1_slpm,f1_stance,f1_str__acc_,f1_str__def,f1_sub__avg_,f1_sub_att,f1_td,f1_td_%,f1_td_acc_,f1_td_avg_,f1_td_def_,f1_total_str,f1_weight,f2_body,f2_clinch,f2_control,f2_distance,f2_dob,f2_ground,f2_head,f2_height,f2_kd,f2_leg,f2_reach,f2_record,f2_rev,f2_sapm,f2_sig_str,f2_sig_str_%,f2_slpm,f2_stance,f2_str__acc_,f2_str__def,f2_sub__avg_,f2_sub_att,f2_td,f2_td_%,f2_td_acc_,f2_td_avg_,f2_td_def_,f2_total_str,f2_weight,gender,year,name_x,winner_cluster,name_y,loser_cluster,win_percentage
0,5:00,Lerone Murphy,Edson Barboza,2024-05-18,Decision - Unanimous,Herb Dean,5,5,featherweight,f1,27 of 34,13 of 17,5:19,205 of 344,"Jul 22, 1991",2 of 3,172 of 308,175.26,0,21 of 22,185.42,Record: 14-0-1,0,2.60,220 of 364,60%,5.01,Orthodox,54%,60%,0.6,0,4 of 6,66%,52%,1.59,46%,259 of 410,65.77,15 of 21,2 of 3,5:19,77 of 239,"Jan 21, 1986",0 of 0,38 of 189,180.34,0,26 of 32,190.50,Record: 24-12-0,0,4.65,79 of 242,32%,4.11,Orthodox,44%,56%,0.1,0,0 of 0,0,50%,0.45,72%,79 of 242,65.77,M,2024,Lerone Murphy,4,Edson Barboza,4,50.0
1,1:30,Khaos Williams,Carlston Harris,2024-05-18,KO/TKO,Dan Miragliotta,1,3,welterweight,f1,2 of 4,0 of 0,0:01,11 of 19,"Mar 30, 1994",1 of 1,5 of 9,182.88,1,5 of 7,195.58,Record: 15-3-0,0,5.42,12 of 20,60%,5.81,Orthodox,39%,42%,0.0,0,0 of 0,0,0%,0.00,80%,12 of 20,77.11,1 of 2,0 of 0,0:01,14 of 30,"Jul 09, 1987",0 of 0,10 of 25,182.88,0,3 of 3,193.04,Record: 19-6-0,0,2.66,14 of 30,46%,3.18,Orthodox,49%,52%,0.8,0,0 of 0,0,29%,1.97,55%,14 of 30,77.11,M,2024,Khaos Williams,4,Carlston Harris,2,52.0
2,5:00,Themba Gorimbo,Ramiz Brahimaj,2024-05-18,Decision - Unanimous,Mark Smith,3,3,welterweight,f1,31 of 35,35 of 38,10:13,6 of 10,"Jan 23, 1991",2 of 4,9 of 13,185.42,0,3 of 4,195.58,Record: 13-4-0,1,1.08,43 of 52,82%,2.53,Orthodox,69%,46%,0.0,0,5 of 6,83%,58%,4.04,78%,110 of 134,77.11,3 of 4,3 of 4,10:13,0 of 2,"Nov 17, 1992",0 of 0,0 of 2,177.80,0,0 of 0,182.88,Record: 10-5-0,0,3.69,3 of 6,50%,1.74,Orthodox,41%,45%,1.6,0,1 of 8,12%,35%,1.61,44%,15 of 20,77.11,M,2024,Themba Gorimbo,2,Ramiz Brahimaj,2,50.0
3,2:47,Adrian Yanez,Vinicius Salvador,2024-05-18,KO/TKO,Chris Tognoni,1,3,bantamweight,f1,1 of 2,0 of 0,0:17,15 of 28,"Nov 29, 1993",21 of 28,27 of 45,170.18,1,8 of 9,177.80,Record: 17-5-0,0,5.75,36 of 56,64%,6.51,Orthodox,41%,57%,0.0,0,0 of 0,0,0%,0.00,100%,36 of 58,61.23,6 of 6,0 of 0,0:17,17 of 34,"Jul 24, 1996",0 of 0,6 of 20,170.18,0,5 of 8,177.80,Record: 14-7-0,0,6.50,17 of 34,50%,5.48,Southpaw,43%,48%,0.0,0,0 of 0,0,11%,0.36,86%,17 of 34,61.23,M,2024,Adrian Yanez,4,Vinicius Salvador,4,50.0
4,4:12,Angela Hill,Luana Pinheiro,2024-05-18,Submission,Herb Dean,2,3,strawweight,f1,9 of 15,3 of 3,3:06,21 of 71,"Jan 12, 1985",4 of 5,19 of 64,160.02,0,0 of 0,162.56,Record: 17-13-0,1,4.86,28 of 79,35%,5.38,Orthodox,49%,61%,0.1,1,2 of 6,33%,33%,0.80,76%,39 of 91,52.16,3 of 6,2 of 6,3:06,15 of 54,"Nov 18, 1992",0 of 0,11 of 51,157.48,0,3 of 3,157.48,Record: 11-3-0,0,4.17,17 of 60,28%,3.57,Orthodox,40%,63%,0.2,0,1 of 7,14%,33%,2.24,66%,25 of 77,52.16,M,2024,Angela Hill,4,Luana Pinheiro,4,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7284,0:39,Jerry Bohlander,Nick Sanzo,1997-02-07,Submission,John McCarthy,1,1,lightweight,f1,0 of 0,2 of 2,0,0 of 2,"Feb 12, 1974",0 of 0,2 of 3,180.34,0,0 of 1,185.29,Record: 11-4-0,0,0.00,2 of 4,50%,0.00,Orthodox,0%,0%,0.0,1,1 of 1,100%,0%,0.00,0%,3 of 5,90.26,0 of 0,0 of 0,0,0 of 0,NaN,0 of 0,0 of 0,175.26,0,0 of 0,179.62,Record: 1-1-0,0,0.00,0 of 0,0,0.00,Orthodox,0%,0%,0.0,0,0 of 1,0%,0%,0.00,0%,0 of 0,86.18,M,1997,Jerry Bohla